# Comparing randomized search and grid search for hyperparameter estimation

Adapted from http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html

Compare randomized search and grid search for optimizing hyperparameters of a random forest. All parameters that influence the learning are searched simultaneously (except for the number of estimators, which poses a time / quality tradeoff).

The randomized search and the grid search explore exactly the same space of parameters. The result in parameter settings is quite similar, while the run time for randomized search is drastically lower.

The performance is slightly worse for the randomized search, though this is most likely a noise effect and would not carry over to a held-out test set.

Note that in practice, one would not search over this many different parameters simultaneously using grid search, but pick only the ones deemed most important.

In [1]:
using ScikitLearn
using PyCall: @pyimport

@pyimport scipy.stats as stats
using ScikitLearn.GridSearch: GridSearchCV, RandomizedSearchCV

@sk_import datasets: load_digits
@sk_import ensemble: RandomForestClassifier

INFO: Recompiling stale cache file /Users/cedric/.julia/lib/v0.5/ScikitLearn.ji for module ScikitLearn.


In [2]:
# get some data
digits = load_digits()
X, y = digits["data"], digits["target"]

# build a classifier
clf = RandomForestClassifier(n_estimators=20, random_state=2)

# Utility function to report best scores
function report(grid_scores, n_top=3)
    top_scores = sort(grid_scores, by=x->x.mean_validation_score, rev=true)[1:n_top]
    for (i, score) in enumerate(top_scores)
        println("Model with rank:$i")
        @printf("Mean validation score: %.3f (std: %.3f)\n",
                score.mean_validation_score,
                std(score.cv_validation_scores))
        println("Parameters: $(score.parameters)")
        println("")
    end
end

sampler(a, b) = stats.randint(a, b)
#sampler(a, b) = DiscreteUniform(a, b-1)  TODO

# specify parameters and distributions to sample from
param_dist = Dict("max_depth"=> [3, nothing],
                  "max_features"=> sampler(1, 11),
                  "min_samples_split"=> sampler(2, 11),
                  "min_samples_leaf"=> sampler(1, 11),
                  "bootstrap"=> [true, false],
                  "criterion"=> ["gini", "entropy"])

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_dist,
                                   n_iter=n_iter_search, random_state=MersenneTwister(42))

start = time()
fit!(random_search, X, y)
@printf("RandomizedSearchCV took %.2f seconds for %d candidates, parameter settings.\n", (time() - start), n_iter_search)

report(random_search.grid_scores_)

# use a full grid over all parameters
param_grid = Dict("max_depth"=> [3, nothing],
                  "max_features"=> [1, 3, 10],
                  "min_samples_split"=> [2, 3, 10],  # min_samples_split=1 became an error in 0.18.1
                  "min_samples_leaf"=> [1, 3, 10],   
                  "bootstrap"=>[true, false],
                  "criterion"=> ["gini", "entropy"])

# run grid search
grid_search = GridSearchCV(clf, param_grid)

start = time()
fit!(grid_search, X, y)

@printf("GridSearchCV took %.2f seconds for %d candidate parameter settings.\n",
time() - start, length(grid_search.grid_scores_))

report(grid_search.grid_scores_)

RandomizedSearchCV took 8.45 seconds for 20 candidates, parameter settings.
Model with rank:1
Mean validation score: 0.927 (std: 0.012)
Parameters: Dict{Symbol,Any}(Pair{Symbol,Any}(:max_features,6),Pair{Symbol,Any}(:bootstrap,false),Pair{Symbol,Any}(:min_samples_split,7),Pair{Symbol,Any}(:max_depth,nothing),Pair{Symbol,Any}(:criterion,"gini"),Pair{Symbol,Any}(:min_samples_leaf,4))

Model with rank:2
Mean validation score: 0.922 (std: 0.007)
Parameters: Dict{Symbol,Any}(Pair{Symbol,Any}(:max_features,4),Pair{Symbol,Any}(:bootstrap,false),Pair{Symbol,Any}(:min_samples_split,8),Pair{Symbol,Any}(:max_depth,nothing),Pair{Symbol,Any}(:criterion,"entropy"),Pair{Symbol,Any}(:min_samples_leaf,6))

Model with rank:3
Mean validation score: 0.916 (std: 0.023)
Parameters: Dict{Symbol,Any}(Pair{Symbol,Any}(:max_features,10),Pair{Symbol,Any}(:bootstrap,false),Pair{Symbol,Any}(:min_samples_split,8),Pair{Symbol,Any}(:max_depth,nothing),Pair{Symbol,Any}(:criterion,"gini"),Pair{Symbol,Any}(:min_samples_

If `refit=true`, `GridSearchCV` and `RandomizedSearchCV` behave like the best estimator they found.

In [3]:
@show predict(random_search, X)[1:10]
@show predict_proba(random_search, X)[1:10]
@show score(random_search, X, y)

(predict(random_search,X))[1:10] = [0,1,2,3,4,5,6,7,8,9]
(predict_proba(random_search,X))[1:10] = [1.0,0.0,0.02,0.0,0.144167,0.0,0.0125,0.0,0.0,0.0]
score(random_search,X,y) = 0.9994435169727324


0.9994435169727324